In [ ]:
# Put all import statements in this cell
import pandas as pd
from sklearn.linear_model import LinearRegression
import re
import numpy as np
from pathlib import Path
from sys import exit
import gc
from manager import program_sleep

In [ ]:
def file_valid(path: Path) -> bool:
    file_name_check = path.name
    valid_extensions = ('.xls', '.xlsx', 'xlsb', '.xlsm')
    return path.is_file() and str(file_name_check).endswith(valid_extensions)

In [ ]:
# Set this to false if you don't want any garbage collecting
collect_garbage = True

def garbage_collector(data: any) -> None:
    if not collect_garbage or not data:
        return 
    
    for d in data:
        if d in globals():
            del globals()[d]
    collected = gc.collect()
    print(f'This is what was collected {collected}')

In [1]:
#WORKING FOR CONVERTING TO YEARS
output_excel_file_save = ''

def convert_years_from_date_cells(excel_file_path_convert, sheet_name_convert) -> None:
    
    df_convert = pd.read_excel(excel_file_path_convert, sheet_name=sheet_name_convert)

    # Define a regular expression pattern to match "Dec. 31, YYYY"
    year_pattern = r'Dec\. 31, (\d{4})'

    # Iterate through the DataFrame and update the cells to extract the year
    for col in df_convert.columns:
        for idx, value in enumerate(df_convert[col]):
            if isinstance(value, str):
                year_match = re.search(year_pattern, value)
                if year_match:
                    year_value = year_match.group(1)
                    df_convert.at[idx, col] = year_value

    # Save the modified DataFrame back to an Excel file
    program_sleep(1.5)
    
    output_excel_file_save = input('Enter the name of the file you want to save the new data: ')
    df_convert.to_excel(output_excel_file_save, index=False)
    
    program_sleep(2.5)

# Call the function with the path to your Excel file
#excel_file_path = '/Users/evan/Desktop/chatbot/DataPriming3/MVST_2022_FY_RUNINNG.xlsx'
#sheet_name = 'CONSOLIDATED STATEMENTS OF OPERATIONS'
excel_file_path = input('Enter the path to your excel file you want to use to convert the years: ')
program_sleep(1.2)
sheet_name = input('Enter the name of the sheet you want to use in the function: ')
convert_years_from_date_cells(excel_file_path, sheet_name)

def get_saved_file() -> str:
    return output_excel_file_save

In [ ]:
#WORKING YEARS FOR HEADERS
# Load your DataFrame from the Excel file

excel_file = get_saved_file()
df = pd.read_excel(excel_file, header=None)

# Set the row with "2022," "2021," and "2020" as the column headers
df.columns = df.iloc[1]

# Drop the rows that you don't need
df = df.drop([0, 1])

# Reset the index
df.reset_index(drop=True, inplace=True)

# Save the modified DataFrame back to an Excel file
#output_excel_file = 'output_file_with_updated_headers.xlsx'
output_excel_file = input('Enter the name of the file you want to save the data as: ')
df.to_excel(output_excel_file, index=False)

def headers_saved_file() -> str:
    return output_excel_file

In [3]:
#WORKS PERFECTLY

def calculate_year_over_year_growth(df_growth):
    years = df_growth.columns

    for i in range(len(years) - 1, 0, -1):
        current_year_growth = years[i]
        prev_year = years[i - 1]

        # Calculate YOY growth, handle potential errors (division by zero)
        growth = ((df_growth[prev_year] - df_growth[current_year_growth]) / df_growth[current_year_growth])

        # Handle potential errors (division by zero, infinite values, and NaN)
        growth = growth.replace([np.inf, -np.inf, np.nan], np.nan)

        # Round the growth to one decimal place
        growth = growth.round(4)

        # Format as a percentage
        growth = (growth * 100).apply(lambda x: f"{x:.1f}%" if not np.isnan(x) else '')

        df_growth.insert(i, f"{current_year_growth} - {prev_year} YOY Growth", growth)

    return df_growth

# Load your DataFrame from the Excel file
excel_file = headers_saved_file()
df_data = pd.read_excel(excel_file, index_col=0, na_values="")

# Clean the data and convert to numeric
df_data = df_data.replace({'\$': '', ',': ''}, regex=True).apply(pd.to_numeric, errors='coerce')

# Calculate YOY growth and add columns
df_data = calculate_year_over_year_growth(df_data)

# Save the modified DataFrame back to an Excel file
#output_excel_file = 'output_file_with_growth10.xlsx'
output_excel_file = input('Enter the name of the file you want to save it as: ')
df_data.to_excel(output_excel_file)

print(df_data)

FileNotFoundError: [Errno 2] No such file or directory: 'output_file_with_updated_headers.xlsx'

In [ ]:
### Concat to prime
df_path = input('Enter the path of the file you want to read. Make sure it is an excel file: ')

df = pd.read_excel(df_path)
df = df.iloc[0:2]
df = df.drop(df.index[0])

df_name = input('Enter the name of the excel file to save the data: ')
df.to_excel(df_name)

In [ ]:
### Getting sector regression data

def convert_percentage_to_float(perc) -> float:
    try:
        return float(perc.strip('%')) / 100.0
    except FloatingPointError:
        return np.nan
    

primed_path = input('Enter the path of the excel file you want to prime the data for: ')
primed_df = pd.read_excel(primed_path)  # Update the path to your file
returns_path = input('Enter the path to the excel file you want to get a return table for: ')
returns_df = pd.read_excel(returns_path)  # Update the path to your file

growth_column = input('Enter the growth column for your primed data excel file: ')
yoy_growth_cols = [col for col in primed_df.columns if re.match(r'\d{4} - \d{4}', col)]
growth_years = [re.findall(r'(\d{4}) - \d{4}', col)[0] for col in yoy_growth_cols]

for col in yoy_growth_cols:
    primed_df[col] = primed_df[col].apply(convert_percentage_to_float)

return_column = input('Enter the name of the column you want to use for your return excel file: ')
returns_df[return_column] = returns_df[return_column].astype(str)

merged_data = pd.DataFrame()

for growth_col, year in zip(yoy_growth_cols, growth_years):
    yoy_growth_data = primed_df[['Unnamed: 0', growth_col]]
    yoy_growth_data = yoy_growth_data.rename(columns={'Unnamed: 0': 'Category', growth_col: growth_column})
    yoy_growth_data[return_column] = year

    merged_year_data = pd.merge(yoy_growth_data, returns_df, on='Year')
    merged_data = pd.concat([merged_data, merged_year_data])

print("Column names in merged_data:", merged_data.columns)

merged_data = merged_data.dropna(subset=['YOY Growth'])


X = merged_data[returns_df.columns.drop(return_column)]
y = merged_data[growth_column]

model = LinearRegression()
model.fit(X, y)

coefficients = model.coef_
intercept = model.intercept_

print("Linear Regression Coefficients:")
for feature, coef in zip(X.columns, coefficients):
    print(f"    {feature}: {coef}")
print(f"Intercept: {intercept}")



In [ ]:
#Get it down to annual return table


def get_return_table():
    return_table_path = input('Enter the path to the return table excel file: ')
    sheet_name_path = input('Enter the name of the sheet you want to use for the return table: ')
    df_table = pd.read_excel(return_table_path, sheet_name_path, header= None)
    return_table = df_table.iloc[17:114, :7]
    return_table.reset_index(drop=True, inplace=True)
    return return_table

df = get_return_table()
return_table_name = input('Enter the name of the return table excel file you want to save it as: ')
df.to_excel(return_table_name)

In [ ]:
''' This will actually give the stock price projection'''


# Load the financial data from the spreadsheet

# file_path = '/Users/evan/Desktop/chatbot/Regression Testing/venv/bin/MergeManipulationsFMP/Components/MVST_2022_FY_RUNINNG.xlsx'  # Update this path
file_path = input('Enter the path to the excel file you want to use to get the stock price projection: ')
# balance_sheet_sheet_name = 'CONSOLIDATED BALANCE SHEETS'
balance_sheet_sheet_name = input('Enter the balance sheet for this excel file: ')
balance_sheet = pd.read_excel(file_path, sheet_name=balance_sheet_sheet_name)

# Extract 'Cash and cash equivalents' and 'Notes payable' for 2022
cash_and_equivalents_2022 = balance_sheet.iloc[1, 1]  # Update row index if needed
notes_payable_2022 = balance_sheet.iloc[19, 1]  # Update row index if needed

# Calculate net debt for 2022
net_debt = notes_payable_2022 - cash_and_equivalents_2022

# Constants - Replace these with your specific assumptions
REVENUE_GROWTH_RATE = 0.15
OPERATING_MARGIN = 0.20
TAX_RATE = 0.21
WACC = 0.08
TERMINAL_GROWTH_RATE = 0.02
FORECAST_PERIOD = 10  
num_shares = 303279188

def load_financial_data(file_path_load, sheet_name_load='CONSOLIDATED STATEMENTS OF OPERATIONS'):
    income_statement_load = pd.read_excel(file_path_load, sheet_name=sheet_name_load)
    return income_statement_load

def forecast_cash_flows(income_statement_flows, year_cash_flows: int) -> float:
    last_year_revenue = income_statement_flows.iloc[2, 1]
    forecasted_revenue = last_year_revenue * (1 + REVENUE_GROWTH_RATE) ** year_cash_flows
    forecasted_ebit = forecasted_revenue * OPERATING_MARGIN
    # forecasted_tax = forecasted_ebit * TAX_RATE
    forecasted_ebitda = forecasted_ebit * (1.0 - TAX_RATE)
    return forecasted_ebitda

def calculate_price_per_year(income_statement_calculate, year_per: int) -> float:
    forecasted_ebitda = forecast_cash_flows(income_statement_calculate, year_per)
    discounted_cash_flow = forecasted_ebitda / (1 + WACC) ** year_per
    terminal_value = discounted_cash_flow * (1 + TERMINAL_GROWTH_RATE) / (WACC - TERMINAL_GROWTH_RATE)
    present_value_terminal = terminal_value / (1 + WACC) ** FORECAST_PERIOD
    enterprise_value = discounted_cash_flow + present_value_terminal
    equity_value = enterprise_value - net_debt
    price_per_share = (equity_value / num_shares) * 1000.0  
    return price_per_share

income_statement = load_financial_data(file_path)

predicted_prices = []
current_year = 2022
for year_main in range(1, FORECAST_PERIOD + 1):
    price = calculate_price_per_year(income_statement, year_main)
    predicted_prices.append(price)

df_predicted_prices = pd.DataFrame({
    'Year': [current_year + i for i in range(FORECAST_PERIOD)],
    'Predicted Share Price': predicted_prices
})

print(df_predicted_prices)
# current_year = 2022


In [ ]:
'''This is the incomplete part because it's connected to the competitor data'''


# import pandas as pd
# import numpy as np 


# ### These will all be separate DFs with the daily closing prices of these companies

# OG_COMPANY = 'TICKER HERE'
# COMP1 = 'TICKER HERE'
# COMP2 = 'TICKER HERE'
# COMP3 = 'TICKER HERE'
# COMP4 = 'TICKER HERE'
# COMP5 = 'TICKER HERE'
# COMP6 = 'TICKER HERE'
# COMP7 = 'TICKER HERE'
# COMP8 = 'TICKER HERE'
# COMP9 = 'TICKER HERE'
# COMP10 = 'TICKER HERE'

# combined_df = pd.merge(OG_COMPANY, COMP1, on='Date', how='outer')
# combined_df = pd.merge(combined_df, COMP2, on='Date', how='outer')
# combined_df = pd.merge(combined_df, COMP3, on='Date', how='outer')
# combined_df = pd.merge(combined_df, COMP4, on='Date', how='outer')
# combined_df = pd.merge(combined_df, COMP5, on='Date', how='outer')
# combined_df = pd.merge(combined_df, COMP6, on='Date', how='outer')
# combined_df = pd.merge(combined_df, COMP7, on='Date', how='outer')
# combined_df = pd.merge(combined_df, COMP8, on='Date', how='outer')
# combined_df = pd.merge(combined_df, COMP9, on='Date', how='outer')
# combined_df = pd.merge(combined_df, COMP10, on='Date', how='outer')


def reverse_dataframe(df_reverse):
    """
    Reverse the DataFrame so that the most recent dates are first.
    """
    return df_reverse.iloc[::-1].reset_index(drop=True)

def convert_to_numeric(df_conversion, date_column='Date'):
  """
  Convert non-numeric values in specified columns to NaN (except for the first row).
  """
  for col in df_conversion.columns:
    if col != date_column:
      df_conversion[col] = pd.to_numeric(df_conversion[col][1:], errors='coerce')
  return df_conversion

def calculate_pct_change(df_calculate, periods) -> int:
    """
    Calculate the percentage change for the given number of periods,
    starting from the second row, and raise an error if non-numeric data is found.
    """
    if not pd.api.types.is_numeric_dtype(df_calculate.iloc[:, 1:].dtypes):
        raise TypeError(f"Non-numeric data found in columns: {df_calculate.columns[1:]}. Please check and rectify before calculating percentage change.")
    return df_calculate.iloc[1:].pct_change(periods) * 100

def calculate_weekly_change(df_weekly) -> int:
    """
    Calculate the weekly percentage change for the 'MVST Close' column after data validation.
    """
    return calculate_pct_change(df_weekly[["Date", "MVST Close"]], 5)

# Load your DataFrame (replace 'file_path' with the actual path)
# file_path = '/Users/evan/Desktop/chatbot/Regression Testing/venv/bin/MergeManipulationsFMP/Components/MVST_daily.xlsx'
file_path = input('Enter the file path of the excel file where there is the competitor data: ')
df = pd.read_excel(file_path)
df_column = input('Enter the column from this competitor data you want to use: ')
df[df_column] = pd.to_numeric(df[df_column], errors='raise', downcast='float')


# Convert 'MVST Close' column to numeric (excluding headers)
df = convert_to_numeric(df.copy())

# Reverse the DataFrame for easier analysis
reversed_df = reverse_dataframe(df)

# Calculate weekly percentage change for 'MVST Close'
weekly_change = calculate_weekly_change(reversed_df)

# Print weekly percentage change (you can adjust this to your needs)
print(f"Weekly Percentage Change for {df_column}:")
print(weekly_change[df_column])

# (Optional) Print information for debugging purposes
print("\nData types after conversion:")
print(df.dtypes)

print("\nFirst few rows of data:")
print(df.head())

df[df_column].describe()